In [1]:
#DOWNLOADING EDITS FROM WIKIPEDIA
import mwclient
import time

#Init a class that enables us to work with a specific wiki site
site = mwclient.Site("en.wikipedia.org")

#Which page we want to look at
page = site.pages["Bitcoin"]

In [2]:
#Fetch our list of revisions from wikipedia about Bitcoin from "https://en.wikipedia.org/w/index.php?title=Bitcoin&action=history"
revs = list(page.revisions())

In [3]:
#Revision is an ordered dictionary
revs[0]

OrderedDict([('revid', 1187285167),
             ('parentid', 1187283988),
             ('user', 'A455bcd9'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=11, tm_mday=28, tm_hour=8, tm_min=33, tm_sec=26, tm_wday=1, tm_yday=332, tm_isdst=-1)),
             ('comment', "We don't do a section for one paragraph")])

In [4]:
#Reorder the edits by sort according to timestamp
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
#FINDING THE SENTIMENT OF PAGE EDITS
#Finding the number of edits per day and sentiment of the edits
from transformers import pipeline

#sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis")

#function to find sentiment of a piece of text
def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"] #Label 0 to 1 depending on strength of sentiment
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
#Testing our sentiment function
find_sentiment("I love you")

0.9998656511306763

In [8]:
#Testing our sentiment function
find_sentiment("I hate you")

-0.9991129040718079

In [9]:
#Process each revision to count how many edits per day and average sentiment per day
#Creating a big dictionary called "edits"
edits = {} 

for rev in revs:
    #date = time.strftime("Y%-%m-%d", rev["timestamp"])
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    #extract the comment and pass it through sentiment analysis
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [10]:
#Summarize the sentiment into pandas df later and merge it with market data
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        
        #Finding % of comments that have negative sentiments
        edits[key]["neg_sentiments"] = len([s for s in edits[key]["sentiments"] if s <0]) / len(edits[key]["sentiments"])

    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiments"] = 0

    del edits[key]["sentiments"]

In [11]:
#Displaying our dictionary
#Each key is a date, inside there are "edit_count", "sentiment" and "neg_sentiment"
edits

{'2009-03-08': {'edit_count': 4,
  'sentiment': -0.550525039434433,
  'neg_sentiments': 0.75},
 '2009-08-05': {'edit_count': 1,
  'sentiment': 0.7481209635734558,
  'neg_sentiments': 0.0},
 '2009-08-06': {'edit_count': 2,
  'sentiment': 0.9957457184791565,
  'neg_sentiments': 0.0},
 '2009-08-14': {'edit_count': 1,
  'sentiment': 0.9300208687782288,
  'neg_sentiments': 0.0},
 '2009-10-13': {'edit_count': 2,
  'sentiment': -0.2275010645389557,
  'neg_sentiments': 0.5},
 '2009-11-18': {'edit_count': 1,
  'sentiment': 0.883950412273407,
  'neg_sentiments': 0.0},
 '2009-12-08': {'edit_count': 1,
  'sentiment': -0.9869275689125061,
  'neg_sentiments': 1.0},
 '2009-12-17': {'edit_count': 1,
  'sentiment': -0.9975171089172363,
  'neg_sentiments': 1.0},
 '2010-02-23': {'edit_count': 1,
  'sentiment': -0.9994946718215942,
  'neg_sentiments': 1.0},
 '2010-03-18': {'edit_count': 1,
  'sentiment': 0.8758774995803833,
  'neg_sentiments': 0.0},
 '2010-04-13': {'edit_count': 4,
  'sentiment': 0.844355

In [12]:
#CONVERTING SENTIMENT DATA INTO A PANDAS DATAFRAME DF
import pandas as pd

#Take in dictionary
edits_df = pd.DataFrame.from_dict(edits, orient="index")
edits_df

,edit_count,sentiment,neg_sentiments
2009-03-08,4,-0.550525,0.750000
2009-08-05,1,0.748121,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930021,0.000000
2009-10-13,2,-0.227501,0.500000
...,...,...,...
2023-11-24,89,-0.122848,0.539326
2023-11-25,7,0.080698,0.428571
2023-11-26,6,-0.667277,0.833333
2023-11-27,3,-0.476871,0.666667


In [13]:
#Convert index to pandas date time index using to_datetime index
edits_df.index = pd.to_datetime(edits_df.index)

In [14]:
#Data provided has a lot of gaps between dates as wikipedia is not updated everyday, but Bitcoin is traded every second
#Merge all data together where there is no missing value
from datetime import datetime

dates = pd.date_range(start="2009-03-08", end=datetime.today())

In [15]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2023-11-19', '2023-11-20', '2023-11-21', '2023-11-22',
               '2023-11-23', '2023-11-24', '2023-11-25', '2023-11-26',
               '2023-11-27', '2023-11-28'],
              dtype='datetime64[ns]', length=5379, freq='D')

In [16]:
#Merge dates with our existing edits_df
#Fill in 0 with days that has no data
edits_df = edits_df.reindex(dates, fill_value=0)

In [17]:
#We can now see all the days including days without edits/changes in wikipedia
edits_df

,edit_count,sentiment,neg_sentiments
2009-03-08,4,-0.550525,0.750000
2009-03-09,0,0.000000,0.000000
2009-03-10,0,0.000000,0.000000
2009-03-11,0,0.000000,0.000000
2009-03-12,0,0.000000,0.000000
...,...,...,...
2023-11-24,89,-0.122848,0.539326
2023-11-25,7,0.080698,0.428571
2023-11-26,6,-0.667277,0.833333
2023-11-27,3,-0.476871,0.666667


In [18]:
#Use rolling average to find out how many edits where there in last months
#Enables us to know how much interest there was in bitcoin in last months
rolling_edit = edits_df.rolling(30).mean()

In [19]:
#Many rows at beginning will have missing values(NaN) because we do not have 30days of data for those days
rolling_edit

,edit_count,sentiment,neg_sentiments
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2023-11-24,16.266667,-0.127673,0.276255
2023-11-25,16.466667,-0.158154,0.290541
2023-11-26,16.666667,-0.180397,0.318319
2023-11-27,16.766667,-0.196292,0.340541


In [20]:
#Drop all the rows that are empty
rolling_edit = rolling_edit.dropna()

In [21]:
rolling_edit

,edit_count,sentiment,neg_sentiments
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2023-11-24,16.266667,-0.127673,0.276255
2023-11-25,16.466667,-0.158154,0.290541
2023-11-26,16.666667,-0.180397,0.318319
2023-11-27,16.766667,-0.196292,0.340541


In [22]:
#Write to csv
rolling_edit.to_csv("wikipedia_edits.csv")